In [1]:
# @title **(1) Configuration for Kaggle Kernels**

# @markdown Configure the environment for Kaggle Kernels.
# @markdown - Mode: Operation mode (no persistent storage in Kaggle).
# @markdown - Data Folder: Folder for training data (path will be relative to the input data directory).
# @markdown - Output Folder: Folder for output (path will be relative to the working directory).
# @markdown - Notification: Alert with a sound when the cell execution is complete.

from IPython.display import Audio, display, clear_output, HTML
import ipywidgets as widgets
import os
import torch

def play_notification_sound(url):
    """Play notification sound."""
    display(Audio(url=url, autoplay=True))

# Configuration variables
mode = "temporary"  # Only temporary mode available in Kaggle
data_dir = "../input/your-dataset-name"  # Replace with your dataset path
output_dir = "./output"  # Output directory in the working directory
enable_notification = "on"  # Notifications can still be configured

def display_forms(device: str):
    """Display configuration forms."""
    clear_output()

    # CSS styling for display
    styles = [
        "<style>.instruction { color:cadetblue;font-weight:300; font-size:1.5rem; line-height:auto }</style>",
        "<style>.runtime_type { color:darkgray;font-weight:300; font-size:1rem; line-height:auto }</style>",
        "<style>.runtime_type_warn { color:red;font-weight:300; font-size:1rem; line-height:auto }</style>",
        "<style>.item_label { color:darkgray;font-weight:700; width:7rem; padding-left:1rem }</style>"
    ]
    for style in styles:
        display(HTML(style))

    instruction = widgets.Label(value='Configure trainer options', layout={'margin': '10px 0'})
    instruction.add_class("instruction")

    # Device type display
    runtime_type = widgets.Label(value=f"Device: {device}")
    runtime_type.add_class("runtime_type")

    # Widgets for configuration
    notification_widget = widgets.ToggleButtons(options=['on', 'off'], value=enable_notification)
    data_dir_widget = widgets.Text(value=data_dir, placeholder="Data folder path")
    output_dir_widget = widgets.Text(value=output_dir, placeholder="Output folder path")

    # Observers for tracking changes
    notification_widget.observe(lambda change: update_config('enable_notification', change['new']), names='value')
    data_dir_widget.observe(lambda change: update_config('data_dir', change['new']), names='value')
    output_dir_widget.observe(lambda change: update_config('output_dir', change['new']), names='value')

    # Display all widgets
    display(instruction, runtime_type)
    display(widgets.HBox([widgets.Label("Data Folder:"), data_dir_widget]))
    display(widgets.HBox([widgets.Label("Output Folder:"), output_dir_widget]))
    display(widgets.HBox([widgets.Label("Notification:"), notification_widget]))

def update_config(param_name: str, value: str):
    """Update configuration parameter."""
    globals()[param_name] = value

# Determine device
device = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "cpu"

# Display forms
display_forms(device)


Label(value='Configure trainer options', layout=Layout(margin='10px 0'), _dom_classes=('instruction',))

Label(value='Device: Tesla T4', _dom_classes=('runtime_type',))

In [2]:
# @title **(2) Setup Environments for Kaggle Kernels**

from pathlib import Path
import os

# Create work dir
def create_work_dir(data_folder, output_folder):
    # Use the Kaggle file structure
    work_dir = Path("./")  # Base directory in Kaggle
    data_dir = work_dir / data_folder
    output_dir = work_dir / output_folder

    # Create necessary directories
    data_dir.mkdir(parents=True, exist_ok=True)
    output_dir.mkdir(parents=True, exist_ok=True)

    return work_dir, data_dir, output_dir

def install_modules(work_dir):
    print("Installing base modules...")
    print(f"You can see logs at {work_dir}/install_module.log")
    !pip install poetry colab-easy-ui==0.1.115 pyworld==0.3.4 &> {work_dir}/install_module.log

def download_trainer_and_move():
    !git clone https://huggingface.co/fierce-cats/beatrice-trainer
    %cd beatrice-trainer
    !git lfs install
    !git lfs pull

# Define data and output folders
data_folder = "data"
output_folder = "output"

work_dir, data_dir, output_dir = create_work_dir(data_folder, output_folder)

install_modules(work_dir)
download_trainer_and_move()

print("fin.")
if enable_notification == "on":
    play_notification_sound("https://github.com/w-okada/beatrice-trainer-colab/raw/master/resources/voices/complete.wav")


Installing base modules...
You can see logs at ./install_module.log
Cloning into 'beatrice-trainer'...
remote: Enumerating objects: 3091, done.
remote: Counting objects: 100% (3091/3091), done.
remote: Compressing objects: 100% (3086/3086), done.
remote: Total 3091 (delta 30), reused 3022 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3091/3091), 2.45 MiB | 14.43 MiB/s, done.
Resolving deltas: 100% (30/30), done.
Filtering content: 100% (2003/2003), 475.77 MiB | 11.68 MiB/s, done.
/kaggle/working/beatrice-trainer
Updated git hooks.
Git LFS initialized.
fin.


In [3]:
import os

# Define the base path for Kaggle inputs
base_path = '/kaggle/input/'

def list_files_recursively(directory, audio_extensions):
    """Recursively list all files in a directory and its subdirectories."""
    all_files = []
    for dirpath, _, filenames in os.walk(directory):
        for filename in filenames:
            all_files.append(os.path.join(dirpath, filename))
    return all_files

# Automatically find the first uploaded folder
try:
    input_dirs = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

    if input_dirs:
        upload_folder_path = os.path.join(base_path, input_dirs[0])
        print(f"Using folder: {upload_folder_path}")

        # List the files in the folder (recursively)
        print("Listing files in the folder and its subdirectories:")
        audio_extensions = ('.wav', '.mp3')  # Specify relevant audio file formats
        all_files = list_files_recursively(upload_folder_path, audio_extensions)

        total_files = len(all_files)
        audio_files = [f for f in all_files if f.endswith(audio_extensions)]

        print(f"Total files found: {total_files}")
        print(f"Audio files found: {len(audio_files)}")

        for filename in all_files:
            print(filename)
    else:
        print("No folders found in the input directory.")
except Exception as e:
    print(f"An error occurred: {e}")


Using folder: /kaggle/input/Davset
Listing files in the folder and its subdirectories:
Total files found: 10
Audio files found: 10
/kaggle/input/Davset/8_cf63-6ec4-4fe8-8bcb-442a674f345d_(Vocals).wav
/kaggle/input/Davset/9_d32b-23f3-48b4-96a0-ccff2cb403df_(Vocals).wav
/kaggle/input/Davset/6_a182-db7e-46e9-9ebd-2ca07657fd7a_(Vocals).wav
/kaggle/input/Davset/5_6527-8a7a-4f1b-a9fc-b45b04e63e16_(Vocals).wav
/kaggle/input/Davset/3_65b8-cd40-4af3-8ead-9a2be352bcad_(Vocals).wav
/kaggle/input/Davset/1_00b0-182e-4e50-bb25-6e9163974919_(Vocals).wav
/kaggle/input/Davset/10_efc8-f87b-4138-87c4-6c97f8ed058d_(Vocals).wav
/kaggle/input/Davset/7_ccab-1583-404e-8a8d-50fe41c4a111_(Vocals).wav
/kaggle/input/Davset/2_3bc4-da0e-4b70-b394-a8e900826d02_(Vocals).wav
/kaggle/input/Davset/4_446c-0dfc-46b7-ba95-75627dce26b5_(Vocals).wav


In [4]:
# @title **(3-2) Extract Training Data**

# @markdown ***!!! Notification !!!***

# @markdown [en] Please execute this after the "upload training data" progress reaches 100%.
# @markdown [日] "upload training data"の進捗が100%になってから実行してください。

import os
import shutil
from IPython.display import Audio, display  # Correct import for Audio

# Define the base path for Kaggle inputs
base_path = '/kaggle/input/'  # Base path for Kaggle inputs

def list_files_recursively(directory, audio_extensions):
    """Recursively list all audio files in a directory and its subdirectories."""
    all_audio_files = []
    for dirpath, _, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith(audio_extensions):
                all_audio_files.append(os.path.join(dirpath, filename))
    return all_audio_files

# Automatically find the first uploaded folder
input_dirs = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]

if input_dirs:
    upload_folder_path = os.path.join(base_path, input_dirs[0])
    print(f"Using folder: {upload_folder_path}")

    # List audio files in the folder (recursively)
    audio_extensions = ('.wav', '.mp3')  # Specify relevant audio file formats
    audio_files = list_files_recursively(upload_folder_path, audio_extensions)

    total_files = len(audio_files)
    print(f"Total audio files found: {total_files}")

    # Directory to save processed data
    data_dir = '/kaggle/working/extracted_data'  # Directory for processed data
    os.makedirs(data_dir, exist_ok=True)

    # Copy audio files to the new directory
    for audio_file in audio_files:
        shutil.copy(audio_file, data_dir)
        print(f"Copied: {audio_file} to {data_dir}")

else:
    print("No folders found in the input directory.")

# Play notification sound if enabled
def play_notification_sound(url):
    """Play notification sound."""
    display(Audio(url=url, autoplay=True))

if 'enable_notification' in locals() and enable_notification == "on":
    play_notification_sound("https://github.com/w-okada/beatrice-trainer-colab/raw/master/resources/voices/complete.wav")


Using folder: /kaggle/input/Davset
Total audio files found: 10
Copied: /kaggle/input/Davset/8_cf63-6ec4-4fe8-8bcb-442a674f345d_(Vocals).wav to /kaggle/working/extracted_data
Copied: /kaggle/input/Davset/9_d32b-23f3-48b4-96a0-ccff2cb403df_(Vocals).wav to /kaggle/working/extracted_data
Copied: /kaggle/input/Davset/6_a182-db7e-46e9-9ebd-2ca07657fd7a_(Vocals).wav to /kaggle/working/extracted_data
Copied: /kaggle/input/Davset/5_6527-8a7a-4f1b-a9fc-b45b04e63e16_(Vocals).wav to /kaggle/working/extracted_data
Copied: /kaggle/input/Davset/3_65b8-cd40-4af3-8ead-9a2be352bcad_(Vocals).wav to /kaggle/working/extracted_data
Copied: /kaggle/input/Davset/1_00b0-182e-4e50-bb25-6e9163974919_(Vocals).wav to /kaggle/working/extracted_data
Copied: /kaggle/input/Davset/10_efc8-f87b-4138-87c4-6c97f8ed058d_(Vocals).wav to /kaggle/working/extracted_data
Copied: /kaggle/input/Davset/7_ccab-1583-404e-8a8d-50fe41c4a111_(Vocals).wav to /kaggle/working/extracted_data
Copied: /kaggle/input/Davset/2_3bc4-da0e-4b70-b3

In [5]:
import os
import shutil

# Assuming audio_files is a list of your audio file paths
audio_files = list_files_recursively(data_dir, ('.wav', '.mp3'))  # Get all audio files

# Organize files by speaker
for audio_file in audio_files:
    # Here you would extract the speaker name from the filename or use a predefined mapping
    # For demonstration, we'll use a placeholder speaker name
    speaker_name = "speaker1"  # Replace this with actual extraction logic based on filenames
    speaker_dir = os.path.join(data_dir, speaker_name)

    # Create the speaker directory if it doesn't exist
    os.makedirs(speaker_dir, exist_ok=True)

    # Copy the audio file to the respective speaker directory
    shutil.copy(audio_file, speaker_dir)
    print(f"Copied: {audio_file} to {speaker_dir}")

# Now you can check the structure
print("Final structure in data_dir:")
for dirpath, dirnames, filenames in os.walk(data_dir):
    print(f"Directory: {dirpath}, Files: {filenames}")


Copied: /kaggle/working/extracted_data/2_3bc4-da0e-4b70-b394-a8e900826d02_(Vocals).wav to /kaggle/working/extracted_data/speaker1
Copied: /kaggle/working/extracted_data/6_a182-db7e-46e9-9ebd-2ca07657fd7a_(Vocals).wav to /kaggle/working/extracted_data/speaker1
Copied: /kaggle/working/extracted_data/8_cf63-6ec4-4fe8-8bcb-442a674f345d_(Vocals).wav to /kaggle/working/extracted_data/speaker1
Copied: /kaggle/working/extracted_data/1_00b0-182e-4e50-bb25-6e9163974919_(Vocals).wav to /kaggle/working/extracted_data/speaker1
Copied: /kaggle/working/extracted_data/9_d32b-23f3-48b4-96a0-ccff2cb403df_(Vocals).wav to /kaggle/working/extracted_data/speaker1
Copied: /kaggle/working/extracted_data/7_ccab-1583-404e-8a8d-50fe41c4a111_(Vocals).wav to /kaggle/working/extracted_data/speaker1
Copied: /kaggle/working/extracted_data/4_446c-0dfc-46b7-ba95-75627dce26b5_(Vocals).wav to /kaggle/working/extracted_data/speaker1
Copied: /kaggle/working/extracted_data/10_efc8-f87b-4138-87c4-6c97f8ed058d_(Vocals).wav to

In [6]:
# @title **(4) Launch TensorBoard**

import os

# Set the port for TensorBoard
PORT = 6006  # Kaggle uses port 6006 for TensorBoard

# Directory where your TensorBoard logs are saved
output_dir = '/kaggle/working/output'  # Update this to your actual log directory

# Run TensorBoard
os.system(f'tensorboard --logdir={output_dir} --host 0.0.0.0 --port {PORT} &')

print("TensorBoard is launching... You can access it at the URL below.")
print(f"http://localhost:{PORT}/")  # Note: In Kaggle, you usually access this in a different way.


TensorBoard is launching... You can access it at the URL below.
http://localhost:6006/



NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

TensorBoard 2.16.2 at http://0.0.0.0:6006/ (Press CTRL+C to quit)


In [ ]:
# @title **(5) Start Training**

# @markdown [EN]
# @markdown It takes about 4-5 hours on T4 to complete the training.
# @markdown [日]
# @markdown トレーニング完了まで、T4で４～５時間程度かかります。(A100だと1時間強)

import os
import shutil
from IPython.display import Audio, display

# Define the data and output directories
data_dir = '/kaggle/working/extracted_data/'  # Directory containing audio files
output_dir = '/kaggle/working/output/'  # Directory to save training output

# Check if audio files exist before starting training
if os.listdir(data_dir):
    print(f"Starting training with audio files in '{data_dir}'...")

    # Run the training script
    !python beatrice_trainer -d {data_dir} -o {output_dir}

    # Optional notification
    enable_notification = "on"  # Change this to "off" to disable notifications

    if enable_notification == "on":
        def play_notification_sound(url):
            """Play notification sound."""
            display(Audio(url=url, autoplay=True))
        
        play_notification_sound("https://github.com/w-okada/beatrice-trainer-colab/raw/master/resources/voices/complete.wav")
else:
    print(f"No audio files found in '{data_dir}'. Please check the directory.")


In [ ]:
# Run the training script with the resume option
!python beatrice_trainer -d {data_dir} -o {output_dir} --resume


In [4]:
import os
import zipfile
from IPython.display import display, HTML, clear_output

def zip_and_download(selected_folder):
    try:
        # Define the path to the folder to zip
        folder_path = os.path.join(output_dir, selected_folder)
        
        # Create a zip file path
        zip_file_path = f'/kaggle/working/{selected_folder}.zip'
        
        # Create the zip file
        with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, _, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, start=folder_path)
                    zipf.write(file_path, arcname)

        # Check if the zip file was created successfully
        if os.path.exists(zip_file_path):
            clear_output(wait=True)  # Clear previous output
            display(HTML(f"<h4>Your zipped folder is ready:</h4> <a href='{zip_file_path}' target='_blank'>Click here to download {selected_folder}.zip</a>"))
            # Additional instruction for download
            display(HTML("<p>If the link does not work, check the Output section in Kaggle.</p>"))
        else:
            print(f"Error: The zip file '{zip_file_path}' was not created.")

    except Exception as e:
        print(f"Error during compression: {e}")

# Define the output directory
output_dir = '/kaggle/working/output/'  # Adjust if necessary

# Ensure the output directory exists
if os.path.exists(output_dir):
    # List the folders in the output directory
    folders = sorted([name for name in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, name))])
    if folders:
        # Display available folders
        print("Available folders:")
        for idx, folder in enumerate(folders):
            print(f"{idx + 1}: {folder}")
        
        # Prompt user for input
        try:
            selected_index = int(input(f"Enter the number of the folder to download (1-{len(folders)}): ")) - 1
            
            if 0 <= selected_index < len(folders):
                selected_folder = folders[selected_index]
                print(f"Zipping folder: {selected_folder}...")
                zip_and_download(selected_folder)
            else:
                print("Invalid selection.")
        except ValueError:
            print("Please enter a valid number.")
    else:
        print(f"No folders found in '{output_dir}'.")
else:
    print(f"Output directory '{output_dir}' does not exist.")
